<a href="https://colab.research.google.com/github/bozorgpanah/The-Explainable-Machine-Learning-Model-withPrivacy/blob/main/Paper1/MDAV%2BSHAP_Warehouse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install shap #Install SHAP library
#Uploading a dataset from local drive to colab
from google.colab import files
uploaded = files.upload()


Saving housing.csv to housing (1).csv


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
import io

##Reading dataset
df = pd.read_csv(io.BytesIO(uploaded['housing.csv'])) #Dataset's name == housing.csv
# Dataset is now stored in a Pandas Dataframe
df.info()

#Missing Value Treatment -------------------------------------------
# Plot the distribution of total bedrooms
#df['total_bedrooms'].value_counts().plot.bar()

# Imputing missing values in total_bedrooms by median
df['total_bedrooms'].fillna(df['total_bedrooms'].median(), inplace=True)

# now check for missing values in total bedrooms
df.isnull().sum()
# ------------------------------------------------------------------


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

In [ ]:
##Feature Vector and Target Variable
X = df[['longitude','latitude','housing_median_age','total_rooms', 'total_bedrooms','population','households','median_income', 'median_house_value']] #Feature Vector
y = df['median_house_value'] #Target Variable

##Splitting the data into train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

print(X_train,y_train)
print("####################################################################################################")
print(X_test,y_test)

       longitude  latitude  ...  median_income  median_house_value
1989     -119.79     36.73  ...         1.9750             47500.0
256      -122.21     37.77  ...         2.2604            100000.0
7887     -118.04     33.87  ...         6.2990            285800.0
4581     -118.28     34.06  ...         1.7199            175000.0
1993     -119.81     36.73  ...         2.2206             59200.0
...          ...       ...  ...            ...                 ...
13123    -121.26     38.27  ...         4.4125            144600.0
19648    -120.89     37.48  ...         2.9135            159400.0
9845     -121.90     36.58  ...         3.1977            289300.0
10799    -117.93     33.62  ...         5.6315            484600.0
2732     -115.56     32.80  ...         1.3882             69400.0

[14448 rows x 9 columns] 1989      47500.0
256      100000.0
7887     285800.0
4581     175000.0
1993      59200.0
           ...   
13123    144600.0
19648    159400.0
9845     289300.0
10799   

In [ ]:
#MDAV a microagregation method ########################################################fabrik
def dist(x,y):
    return np.linalg.norm(x-y)
    
#Stack arrays in sequence vertically (row wise)
def poprow(arr,i):
    pop = arr[i]
    new_array = np.vstack((arr[:i],arr[i+1:]))
    return new_array,pop

def cluster(X, p, k, dist_to_xr):
    c = [p]
    
    if dist_to_xr == None:
        distances = [dist(v[:-1],p[:-1]) for v in X]
    else:
        distances = dist_to_xr
        
    X = X[np.argpartition(distances, k-1)]
    c.extend(X[:k-1])
    X = X[k-1:]
    
    xc = np.array([p[:-1] for p in c], copy=False, ndmin=2)
    yc = np.array([p[-1] for p in c], copy=False)
    cl = (xc, yc)
    
    return X, cl

def mdav(X, y, k):
    D = np.column_stack((X,y)) #D is a matrix of variables
    clusters = []
    centroids = []
    while len(D) >= 3*k:
        # Centroid
        xm = np.mean(D, axis=0) #xm is an array includes mean in each variables (each columns)
        # Furthest from centroid
        xri = np.argmax([dist(v[:-1],xm[:-1]) for v in D])#Find furthest from the centroid in each vector and it's called xri
        D, xr = poprow(D, xri)
        # Furthest from furthest from centroid
        dist_to_xr = [dist(v[:-1],xr[:-1]) for v in D]
        xsi = np.argmax(dist_to_xr)
        dist_to_xr = dist_to_xr[:xsi]+dist_to_xr[xsi+1:]
        D, xs = poprow(D, xsi) 

        #cluster of xr
        D, c = cluster(D, xr, k, dist_to_xr)
        clusters.append(c)
        #cluster of xs
        D, c = cluster(D, xs, k, None)
        clusters.append(c)
        
    if len(D) >= 2*k and len(D) < 3*k:
        # Centroid
        xm = np.mean(D, axis=0)
        # Furthest from centroid
        xri = np.argmax([dist(v[:-1],xm[:-1]) for v in D])
        D, xr = poprow(D, xri)
        #cluster of xr
        D, c = cluster(D, xr, k, None)
        clusters.append(c)
        
        # rest of points
        xc = np.array([p[:-1] for p in D[:]], copy=False, ndmin=2)
        yc = np.array([p[-1] for p in D[:]], copy=False)
        cl = (xc, yc)
        clusters.append(cl)     
    else:
        # rest of points
        xc = np.array([p[:-1] for p in D[:]], copy=False, ndmin=2)
        yc = np.array([p[-1] for p in D[:]], copy=False)
        cl = (xc, yc)
        clusters.append(cl)
    
    centroids = np.array([np.mean(c[0],axis=0) for c in clusters], copy=False)
    
    return clusters, centroids

In [ ]:
import csv

with open('test.csv','w',newline='') as f:
  fieldnames = ['longitude','latitude','housing_median_age','longitude','latitude','housing_median_age','longitude','latitude']
  thewriter = csv.DictWriter(f, fieldnames=fieldnames)
'''
  thewriter.writeheader()
  for i in range(1,100):
   thewriter.writerow({'longitude':'one','latitude':'two','housing_median_age':'three'})
   thewriter.writerow({'longitude':'four','latitude':'six','housing_median_age':'five'})

files.download("mycsv.csv")
'''

with open('test.csv','w',newline='') as fp:
  fieldnames = ['col_1','col_2','col_3','col_4','col_5','col_6','col_7','col_8']
  a = csv.DictWriter(fp, fieldnames=fieldnames)
  a.writeheader()
  a = csv.writer(fp, delimiter = ',')
  data = [[-1.18440e+02, 3.39800e+01, 2.10000e+01, 1.81320e+04, 5.41900e+03,  7.43100e+03, 4.93000e+03, 5.33590e+00, 5.00001e+05],
          [-1.18420e+02, 3.37500e+01, 2.20000e+01, 1.75910e+04, 2.60400e+03, 6.89700e+03, 2.49200e+03, 8.28310e+00, 5.00001e+05],
          [-1.18880e+02, 3.40200e+01, 1.90000e+01, 1.59900e+04, 2.61100e+03, 5.17500e+03, 2.17300e+03, 7.78480e+00, 5.00001e+05],
          [-1.18670e+02, 3.41600e+01, 1.70000e+01, 1.65440e+04, 2.20600e+03, 6.21400e+03, 2.11800e+03, 9.12280e+00, 5.00001e+05],
          [-1.18800e+02, 3.41900e+01, 4.00000e+00, 1.55720e+04, 2.22200e+03, 5.49500e+03, 2.15200e+03, 8.64990e+00, 5.00001e+05],
          [-1.18260e+02, 3.41800e+01, 3.20000e+01, 1.45560e+04, 2.07700e+03, 5.45900e+03, 2.01700e+03, 8.16570e+00, 5.00001e+05],
          [-1.18380e+02, 3.37700e+01, 2.10000e+01, 1.13530e+04, 1.53700e+03, 4.64900e+03, 1.50400e+03, 9.82100e+00, 5.00001e+05],
          [-1.18510e+02, 3.41600e+01, 2.30000e+01, 1.11540e+04, 1.99500e+03, 4.07600e+03, 1.80900e+03, 5.46090e+00, 5.00001e+05],
          [-1.18340e+02, 3.37800e+01, 2.50000e+01, 1.10160e+04, 1.62600e+03, 4.16800e+03, 1.58400e+03, 8.17820e+00, 5.00001e+05],
          [-1.21890e+02, 3.78200e+01, 4.00000e+00, 1.14440e+04, 1.35500e+03, 3.89800e+03, 1.25700e+03, 1.32949e+01, 5.00001e+05]]

  a.writerows(data)
files.download("test.csv")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#We use smaller K amount
import time
import statistics 
exec_times = []

# Generate clusters for different representativities
#representativity = [0.001,0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 
#                    0.009, 0.01, 0.02, 0.03]
"""
representativity = [0.01,0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09,0.1]

K = [int(len(X)*r) for r in representativity]#K = [int(len(X_train)*r) for r in representativity]
"""
K = 206
i = 0
k = int(20640/K)
clusterings = []
centroids = []
maskedData = []
print(f'k = {k}')

 #for k in K:
start = time.time()
    #print(k, K)
clustering, centroid = mdav(X, y, k)#clustering, centroid = mdav(X_train, y_train, k)
clusterings.append(clustering)
centroids.append(centroid)


with open('clusterings_csv.csv','w',newline='') as fp:
   c = csv.writer(fp, delimiter = ',')    
   for line in clusterings:
    c.writerows(line)
i = 0
with open('centroids_csv.csv','w',newline='') as f:
    z = csv.writer(f, delimiter = ',')
    for line in centroids:
      while i<3:  #k times repeat a centroids instead of each member(related to cluster)
        z.writerows(line)
        i += 1

i = 0
while i<k:
 maskedData.append(centroid) 
 i += 1  

end = time.time()
exec_times.append(end-start)
    #arr_ctr_num.append(len(centroids))
print(f'Time for calculating = {exec_times}')

#Saveing centroids in a file
f_centroids = open("Centroids.txt","wt")
f_centroids.write("\n"+str(centroids)+"\n")
f_clusters = open("Clusers.txt","wt")
f_clusters.write("\n"+str(clusterings)+"\n")
f_maskedData = open("MaskedData.txt","wt")
f_maskedData.write("\n"+str(maskedData)+"\n")
f_maskedData.close()
f_centroids.close()
f_clusters.close()
#files.download("MaskedData.txt")
#files.download("Centroids.txt")
#files.download("Clusers.txt")
files.download("clusterings_csv.csv")
files.download("centroids_csv.csv")
#files.download("yc.csv")

k = 100
Time for calculating = [23.297164916992188]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#MDAV a microagregation method  ###############################################################Version 2
import csv
def dist(x,y):
    return np.linalg.norm(x-y)
    
#Stack arrays in sequence vertically (row wise)
def poprow(arr,i):
    pop = arr[i]
    new_array = np.vstack((arr[:i],arr[i+1:]))
    return new_array,pop

def cluster(X, p, k, dist_to_xr):
    c = [p]
    
    if dist_to_xr == None:
        distances = [dist(v[:-1],p[:-1]) for v in X]
    else:
        distances = dist_to_xr
        
    X = X[np.argpartition(distances, k-1)]
    c.extend(X[:k-1])
    X = X[k-1:]
    
    xc = np.array([p[:-1] for p in c], copy=False, ndmin=2)
    yc = np.array([p[-1] for p in c], copy=False)
    cl = (xc, yc)
    '''
    with open('xc.csv','a',newline='') as fp:
     a = csv.writer(fp, delimiter = ',')    
     a.writerows(xc)
     #files.download("xc.csv")
    with open('yc.csv','a',newline='') as fp:
     b = csv.writer(fp, delimiter = ',')    
     b.writerows(yc)
     #files.download("yc.csv")
    '''
    return X, xc, yc

def mdav(X, y, k):
    D = np.column_stack((X,y)) #D is a matrix of variables
    clusters = []
    centroids = []
    ii = 0
    

    while len(D) >= 3*k:
        # Centroid
        xm = np.mean(D, axis=0) #xm is an array includes mean in each variables (each columns)
        # Furthest from centroid
        xri = np.argmax([dist(v[:-1],xm[:-1]) for v in D])#Find furthest from the centroid in each vector and it's called xri
        D, xr = poprow(D, xri)
        # Furthest from furthest from centroid
        dist_to_xr = [dist(v[:-1],xr[:-1]) for v in D]
        xsi = np.argmax(dist_to_xr)
        dist_to_xr = dist_to_xr[:xsi]+dist_to_xr[xsi+1:]
        D, xs = poprow(D, xsi) 

        #cluster of xr
        D, xc_r, yc_r = cluster(D, xr, k, dist_to_xr)
        clusters.append(xc_r)
        clusters.append(yc_r)
        with open('xc.csv','a',newline='') as fp_xc:
         a = csv.writer(fp_xc, delimiter = ',') 
         a.writerows(xc_r)
        with open('yc.csv','a',newline='') as fp_yc:
         b = csv.writer(fp_yc)#, delimiter = '')
         b.writerows(map(lambda x: [x], yc_r))
        #cluster of xs
         D, xc_r, yc_r = cluster(D, xs, k, None)
         clusters.append(xc_r)
         clusters.append(yc_r)
         with open('xc.csv','a',newline='') as fp_xc:
          a = csv.writer(fp_xc, delimiter = ',') 
          a.writerows(xc_r)
         with open('yc.csv','a',newline='') as fp_yc:
          b = csv.writer(fp_yc)
          b.writerows(map(lambda x: [x], yc_r))
        
    if len(D) >= 2*k and len(D) < 3*k:
        # Centroid
        xm = np.mean(D, axis=0)
        # Furthest from centroid
        xri = np.argmax([dist(v[:-1],xm[:-1]) for v in D])
        D, xr = poprow(D, xri)
        #cluster of xr
        D, xc_r, yc_r = cluster(D, xr, k, None)
        clusters.append(xc_r)
        clusters.append(yc_r)
        with open('xc.csv','a',newline='') as fp_xc:
         a = csv.writer(fp_xc, delimiter = ',') 
         a.writerows(xc_r)
        with open('yc.csv','a',newline='') as fp_yc:
         b = csv.writer(fp_yc)
         b.writerows(map(lambda x: [x], yc_r))
        
        # rest of points
        xc = np.array([p[:-1] for p in D[:]], copy=False, ndmin=2)
        yc = np.array([p[-1] for p in D[:]], copy=False)
        #cl = (xc, yc)
        clusters.append(xc)
        clusters.append(yc)
        with open('xc.csv','a',newline='') as fp_xc:
         a = csv.writer(fp_xc, delimiter = ',') 
         a.writerows(xc)
        with open('yc.csv','a',newline='') as fp_yc:
         b = csv.writer(fp_yc)
         b.writerows(map(lambda x: [x], yc))    
    else:
        # rest of points
        xc = np.array([p[:-1] for p in D[:]], copy=False, ndmin=2)
        yc = np.array([p[-1] for p in D[:]], copy=False)
        #cl = (xc, yc)
        clusters.append(xc)
        clusters.append(yc)
        with open('xc.csv','a',newline='') as fp_xc:
         a = csv.writer(fp_xc, delimiter = ',') 
         a.writerows(xc)
        with open('yc.csv','a',newline='') as fp_yc:
         b = csv.writer(fp_yc)
         b.writerows(map(lambda x: [x], yc))  
    
    centroids = np.array([np.mean(c[0],axis=0) for c in clusters], copy=False)
    files.download("xc.csv")
    #print(k)
    return clusters, centroids

In [ ]:
#We use smaller K amount########################################################fabrik
import time
import statistics 
exec_times = []

# Generate clusters for different representativities
#representativity = [0.001,0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 
#                    0.009, 0.01, 0.02, 0.03]
representativity = [0.01,0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09,0.1]

K = [int(len(X)*r) for r in representativity]#K = [int(len(X_train)*r) for r in representativity]
clusterings = []
centroids = []
maskedData = []
print(f'k = {K}')

for k in K:
    start = time.time()
    #print(k, K)
    clustering, centroid = mdav(X, y, k)#clustering, centroid = mdav(X_train, y_train, k)
    clusterings.append(clustering)
    centroids.append(centroid) 
    i = 0
    while i<k:
     maskedData.append(centroid) 
     i += 1  

    end = time.time()
    exec_times.append(end-start)
    #arr_ctr_num.append(len(centroids))
    
#print(f'Time for calculating = {exec_times}')

#Saveing centroids in a file
f_centroids = open("Centroids.txt","wt")
f_centroids.write("\n"+str(centroids)+"\n")
f_clusters = open("Clusers.txt","wt")
f_clusters.write("\n"+str(clusterings)+"\n")
f_maskedData = open("MaskedData.txt","wt")
f_maskedData.write("\n"+str(maskedData)+"\n")
f_maskedData.close()
f_centroids.close()
f_clusters.close()
files.download("MaskedData.txt")
files.download("Centroids.txt")
files.download("Clusers.txt")
#dr = np.array([4,2,1,0.7,0.6,0.5,0.4,0.3,0.2,0.1,0,0])
#k = np.array([8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48 , 52])

In [ ]:
#MDAV a microagregation method #################################Version 3
import csv
def dist(x,y):
    return np.linalg.norm(x-y)
    
#Stack arrays in sequence vertically (row wise)
def poprow(arr,i):
    pop = arr[i]
    new_array = np.vstack((arr[:i],arr[i+1:]))
    return new_array,pop

def cluster(X, p, k, dist_to_xr):
    c = [p]
    
    if dist_to_xr == None:
        distances = [dist(v[:-1],p[:-1]) for v in X]
    else:
        distances = dist_to_xr
        
    X = X[np.argpartition(distances, k-1)]
    c.extend(X[:k-1])
    X = X[k-1:]
    
    xc = np.array([p[:-1] for p in c], copy=False, ndmin=2)
    yc = np.array([p[-1] for p in c], copy=False)
    cl = (xc, yc)
    '''
    with open('xc.csv','a',newline='') as fp:
     a = csv.writer(fp, delimiter = ',')    
     a.writerows(xc)
     #files.download("xc.csv")
    with open('yc.csv','a',newline='') as fp:
     b = csv.writer(fp, delimiter = ',')    
     b.writerows(yc)
     #files.download("yc.csv")
    '''
    return X, xc, yc

def mdav(X, y, k):
    D = np.column_stack((X,y)) #D is a matrix of variables
    clusters = []
    centroids = []

    with open('xc.csv','a',newline='') as fp_xc:
     a = csv.writer(fp_xc, delimiter = ',')
    with open('yc.csv','a',newline='') as fp_yc:
     b = csv.writer(fp_yc, delimiter = ',')    
     while len(D) >= 3*k:
        # Centroid
        xm = np.mean(D, axis=0) #xm is an array includes mean in each variables (each columns)
        # Furthest from centroid
        xri = np.argmax([dist(v[:-1],xm[:-1]) for v in D])#Find furthest from the centroid in each vector and it's called xri
        D, xr = poprow(D, xri)
        # Furthest from furthest from centroid
        dist_to_xr = [dist(v[:-1],xr[:-1]) for v in D]
        xsi = np.argmax(dist_to_xr)
        dist_to_xr = dist_to_xr[:xsi]+dist_to_xr[xsi+1:]
        D, xs = poprow(D, xsi) 

        #cluster of xr
        D, xc_r, yc_r = cluster(D, xr, k, dist_to_xr)
        clusters.append(xc_r)
        clusters.append(yc_r)
        a.writerows(xc_r)
        b.writerows(yc_r)
        #cluster of xs
        D, xc_r, yc_r = cluster(D, xs, k, None)
        clusters.append(xc_r)
        clusters.append(yc_r)
        a.writerows(xc_r)
        b.writerows(yc_r)
        
     if len(D) >= 2*k and len(D) < 3*k:
        # Centroid
        xm = np.mean(D, axis=0)
        # Furthest from centroid
        xri = np.argmax([dist(v[:-1],xm[:-1]) for v in D])
        D, xr = poprow(D, xri)
        #cluster of xr
        D, xc_r, yc_r = cluster(D, xr, k, None)
        clusters.append(xc_r)
        clusters.append(yc_r)
        a.writerows(xc_r)
        b.writerows(yc_r)
        
        # rest of points
        xc = np.array([p[:-1] for p in D[:]], copy=False, ndmin=2)
        yc = np.array([p[-1] for p in D[:]], copy=False)
        #cl = (xc, yc)
        clusters.append(xc)
        clusters.append(yc)
        a.writerows(xc)
        b.writerows(yc)    
     else:
        # rest of points
        xc = np.array([p[:-1] for p in D[:]], copy=False, ndmin=2)
        yc = np.array([p[-1] for p in D[:]], copy=False)
        #cl = (xc, yc)
        clusters.append(xc)
        clusters.append(yc)
        a.writerows(xc)
        b.writerows(yc)
    
    centroids = np.array([np.mean(c[0],axis=0) for c in clusters], copy=False)
    #files.download("test.csv")
    #print(k)
    return clusters, centroids, a, b